## Work
1. 試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異
2. 請嘗試將 BN 放在 Activation 之前，並比較訓練結果
3. 請於 BN 放在 Input Layer 後，並比較結果

In [1]:
import os
import keras
import itertools
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation
from keras.optimizers import SGD

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
# write function to plot results
def plot_results(results):
    color_bar=['maroon','red','sienna','greenyellow','lime','green','aqua','dodgerblue','mediumblue','slateblue','darkorchid','purple']
    """
    Draw the results
    """
    plt.figure(figsize=(12,9))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train_loss'])), results[cond]['train_loss'], '-', label=cond, color=color_bar[i%12])
        plt.plot(range(len(results[cond]['valid_loss'])), results[cond]['valid_loss'],'--', label=cond, color=color_bar[i%12])
    plt.title('Loss')
    plt.legend()
    plt.show

    plt.figure(figsize=(12,9))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train_acc'])), results[cond]['train_acc'], '-', label=cond, color=color_bar[i%12])
        plt.plot(range(len(results[cond]['valid_acc'])), results[cond]['valid_acc'], '--', label=cond, color=color_bar[i%12])
    plt.title('Accuracy')
    plt.legend()
    plt.show()

In [6]:
## 超參數設定
"""
Set your hyper-parameters
"""
LEARNING_RATE=1e-3
MOMENTUM=0.9
EPOCHS=30
BATCH_SIZE=[2, 16, 32, 128, 256]

## 作業一、試比較有 BN 在 Batch_size = 2, 16, 32, 128, 256 下的差異

In [7]:
# write function to build model with BatchNormalization after each hiddenlayer
def build_mlp(input_shape, output_units=10, num_neurons=[128,64,32]):
    """
    Build your own model
    """
    model=Sequential()
    model.add(Dense(units=input_shape[1], input_dim=input_shape[1],kernel_initializer='normal',activation='relu',name='input'))
    
    for i, n_units in enumerate(num_neurons):
        model.add(Dense(units=n_units,kernel_initializer='normal',activation='relu',name='hidden'+str(i+1)))
        model.add(BatchNormalization())
    model.add(Dense(units=output_units,kernel_initializer='normal',activation='softmax',name='output'))

    return model

In [13]:
results_hw1 = {}
"""
Write your training loop and record results
"""
for batch_size in BATCH_SIZE:
    keras.backend.clear_session()
    
    model=build_mlp(input_shape=x_train.shape)
    model.summary()
    optimizer=SGD(lr=LEARNING_RATE,nesterov=True,momentum=MOMENTUM)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    
    model.fit(x_train,y_train,
              epochs=EPOCHS,
              batch_size=batch_size,
              validation_data=(x_test,y_test),
              shuffle=True)
    
    # collect results
    train_loss=model.history.history['loss']
    valid_loss=model.history.history['val_loss']
    train_acc=model.history.history['acc']
    valid_acc=model.history.history['val_acc']
    
    # construct dictionary
    exp_name_tag = 'exp_%s'%str('batch-size_')+str(batch_size)
        
    results_hw1[exp_name_tag]={'train_loss':train_loss,
                               'valid_loss':valid_loss,
                               'train_acc':train_acc,
                               'valid_acc':valid_acc}

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 3072)              9440256   
_________________________________________________________________
hidden1 (Dense)              (None, 128)               393344    
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
hidden2 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
hidden3 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_3 (Batch (None, 32)                128       
__________

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

"""
Plot results
"""
plot_results(results_hw1)

In [ ]:
## 作業二、將 BN 放在 Activation 之前，並比較訓練結果

In [ ]:
# write function to build model with BatchNormalization before Activation
def build_mlp2(input_shape, output_units=10, num_neurons=[128,64,32]):
    """
    Build your own model
    """
    model=Sequential()
    model.add(Dense(units=input_shape[1], input_dim=input_shape[1],kernel_initializer='normal',activation='relu',name='input'))
    
    for i, n_units in enumerate(num_neurons):
        model.add(Dense(units=n_units,kernel_initializer='normal',name='hidden'+str(i+1)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
    model.add(Dense(units=output_units,kernel_initializer='normal',activation='softmax',name='output'))

    return model

In [ ]:
results_hw2 = {}
"""
Write your training loop and record results
"""
for batch_size in BATCH_SIZE:
    keras.backend.clear_session()
    
    model=build_mlp2(input_shape=x_train.shape)
    model.summary()
    optimizer=SGD(lr=LEARNING_RATE,nesterov=True,=momentum=MOMENTUM)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    
    model.fit(x_train,y_train,
              epochs=EPOCHS,
              batch_size=batch_size,
              validation_data=(x_test,y_test),
              shuffle=True)
    
    # collect results
    train_loss=model.history.history['loss']
    valid_loss=model.history.history['val_loss']
    train_acc=model.history.history['acc']
    valid_acc=model.history.history['val_acc']
    
    # construct dictionary
    exp_name_tag = 'exp_%s'%str('batch-size_')+str(batch_size)
        
    results_hw2[exp_name_tag]={'train_loss':train_loss,
                               'valid_loss':valid_loss,
                               'train_acc':train_acc,
                               'valid_acc':valid_acc}

In [ ]:
"""
Plot results
"""
plot_results(results_hw2)

In [ ]:
## 作業三、 將BN 放在 Input Layer 後，並比較結果

In [ ]:
# write function to build model with BatchNormalization after input
def build_mlp3(input_shape, output_units=10, num_neurons=[128,64,32]):
    """
    Build your own model
    """
    model=Sequential()
    model.add(Dense(units=input_shape[1], input_dim=input_shape[1],kernel_initializer='normal',activation='relu',name='input'))
    model.add(BatchNormalization())
    for i, n_units in enumerate(num_neurons):
        model.add(Dense(units=n_units,kernel_initializer='normal',activation='relu',name='hidden'+str(i+1)))
    model.add(Dense(units=output_units,kernel_initializer='normal',activation='softmax',name='output'))

    return model

In [ ]:
results_hw3 = {}
"""
Write your training loop and record results
"""
for batch_size in BATCH_SIZE:
    keras.backend.clear_session()
    
    model=build_mlp3(input_shape=x_train.shape)
    model.summary()
    optimizer=SGD(lr=LEARNING_RATE,nesterov=True,=momentum=MOMENTUM)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    
    model.fit(x_train,y_train,
              epochs=EPOCHS,
              batch_size=batch_size,
              validation_data=(x_test,y_test),
              shuffle=True)
    
    # collect results
    train_loss=model.history.history['loss']
    valid_loss=model.history.history['val_loss']
    train_acc=model.history.history['acc']
    valid_acc=model.history.history['val_acc']
    
    # construct dictionary
    exp_name_tag = 'exp_%s'%str('batch-size_')+str(batch_size)
        
    results_hw3[exp_name_tag]={'train_loss':train_loss,
                               'valid_loss':valid_loss,
                               'train_acc':train_acc,
                               'valid_acc':valid_acc}

In [ ]:
"""
Plot results
"""
plot_results(results_hw3)